In [1]:
from groq import Groq
import pandas as pd
import re
import os

# Import the data

In [2]:
df = pd.read_csv('../../data_acquisition/data/df_with_salary.csv')

In [3]:
df.head()

,title,company,contract_type,description,min_workload,max_workload,min_salary,max_salary
0,Software Engineer .Net,ELCA Informatik AG,Festanstellung,"Über ELCA Wir sind ELCA, einer der grössten Sc...",80,100,100000,130000
1,Digital Analyst / Web Analyst,Unic AG,Festanstellung,Deine Aufgaben als Digital Analyst bei Unic ...,80,100,90000,113000
2,ICT Supporter 100% (a),Spitex Zürich,Festanstellung,Möchtest du auch etwas bewegen und deine Arbei...,100,100,78000,96000
3,KAUFMÄNNISCHES PRAKTIKUM (100%) - MIT FOKUS KU...,gebana AG,Praktikum,DEINE AUFGABEN IM KUND:INNENSERVICE – EINSATZ ...,100,100,28300,28300
4,System Engineer für Microsoft-Technologien und...,konekkt GmbH,Festanstellung,Wir präsentieren eine faszinierende Karrieremö...,80,100,95000,125000


# Prompt blocks definition
Block 0: Few shot 5 examples with salary
Block 1: Job title
Block 2: company
Block 3: workload (min workload and max workload)
Block 4: Contract type
Block 5: description

# Set up for ussage

In [4]:
client = Groq(
    api_key="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",
)

def generate_message(title, company, min_salary, max_salary, min_workload, max_workload, description):
    return {
        "role": "user",
        "content": f"Das Gehalt für die Position {title} bei der Firma {company} liegt zwischen {min_salary} und {max_salary} mit einer Arbeitszeit von {min_workload} bis {max_workload} Prozent. Hier ist die Beschreibung, was Sie benötigen und tun müssen: {description}"
    }

def extract_salaries(response):
    # General regex pattern - covers most cases
    general_pattern = re.compile(r"(\d{1,3}(?:[',\s]\d{3})*)\s*-\s*(\d{1,3}(?:[',\s]\d{3})*)\s*CHF")
    # Special regex pattern - handles "CHF" before the numbers and potential markdown
    special_pattern = re.compile(r"CHF\s*(\d{1,3}(?:[',\s]\d{3})*)\s*[-–]\s*CHF\s*(\d{1,3}(?:[',\s]\d{3})*)")

    # Try the general pattern first
    matches = general_pattern.findall(response)
    if matches:
        min_salary, max_salary = [int(salary.replace("'", "").replace(",", "").replace(" ", "")) for salary in matches[0]]
        return min_salary, max_salary
    
    # If no matches, try the special pattern
    matches = special_pattern.findall(response)
    if matches:
        min_salary, max_salary = [int(salary.replace("'", "").replace(",", "").replace(" ", "")) for salary in matches[0]]
        return min_salary, max_salary
    
    return None, None

## LLM request with Block 1

In [5]:
# change here for new prompt and message below ------------
results_file = 'data/llm_block1.csv'
min = 'block1_min_salary'
max = 'block1_max_salary'
answer = 'block1_answer'
# Setup and load initial data
if os.path.exists(results_file):
    df = pd.read_csv(results_file)
else:
    df[min] = None  # Initialize columns if starting fresh
    df[max] = None
    df[answer] = None 

# change content here for new prompt and message below ------------
def generate_query(row):
    return {
        "role": "user",
        "content": f"Gib mir nur das Gehalt im Format (\d{1,3}(?:'\d{3})*) und (\d{1,3}(?:'\d{3})*) CHF für die Position {row['title']} in Zürich, Schweiz. Wenn du kein exaktes Gehalt hast gib eine Schätzung an."
    }

def run_model(limit):
    start_index = df[min].last_valid_index() + 1 if df[max].last_valid_index() != None else 0

    
    for index, row in df.iloc[start_index:start_index + limit].iterrows():
        query_message = generate_query(row)
        chat_completion = client.chat.completions.create(
            messages= [query_message], # change message for new prompt ------------
            model="llama3-70b-8192",
            temperature=0.0
        )
        min_salary, max_salary = extract_salaries(chat_completion.choices[0].message.content)
        print(index)
        print(min_salary, max_salary)
        
        
        # Update DataFrame
        df.at[index, min] = min_salary
        df.at[index, max] = max_salary
        df.at[index, answer] = chat_completion.choices[0].message.content
        
    # Save updated DataFrame in chunks or fully, depending on your preference
    df.to_csv(results_file, index=False)

# Testing the function with a limited number of entries
run_model(10000)

0
80000 140000
1
80000 120000
2
80000 110000
3
4500 6000
4
90000 140000
5
120000 180000
6
90000 140000
7
120000 150000
8
80000 110000
9
80000 120000
10
120000 180000
11
80000 120000
12
120000 150000
13
80000 140000
14
120000 180000
15
120000 180000
16
120000 180000
17
90000 140000
18
80000 120000
19
80000 120000
20
90000 140000
21
80000 140000
22
80000 120000
23
90000 120000
24
90000 120000
25
80000 120000
26
90000 120000
27
120000 150000
28
90000 120000
29
90000 120000
30
80000 120000
31
90000 120000
32
90000 140000
33
90000 120000
34
90000 120000
35
120000 150000
36
110000 140000
37
90000 120000
38
90000 120000
39
90000 120000
40
90000 120000
41
120000 150000
42
90000 120000
43
90000 120000
44
120000 150000
45
90000 120000
46
90000 120000
47
90000 120000
48
120000 150000
49
120000 180000
50
120000 180000
51
90000 120000
52
80000 110000
53
90000 120000
54
90000 120000
55
90000 120000
56
90000 120000
57
95000 125000
58
90000 120000
59
95000 125000
60
90000 120000
61
80000 110000
62
120

In [6]:
df_block1 = pd.read_csv('data/llm_block1.csv')
df_block1.head()

,title,company,contract_type,description,min_workload,max_workload,min_salary,max_salary,block1_min_salary,block1_max_salary,block1_answer
0,Software Engineer .Net,ELCA Informatik AG,Festanstellung,"Über ELCA Wir sind ELCA, einer der grössten Sc...",80,100,100000,130000,80000,140000,"Based on national averages and online sources,..."
1,Digital Analyst / Web Analyst,Unic AG,Festanstellung,Deine Aufgaben als Digital Analyst bei Unic ...,80,100,90000,113000,80000,120000,"Based on national averages and online sources,..."
2,ICT Supporter 100% (a),Spitex Zürich,Festanstellung,Möchtest du auch etwas bewegen und deine Arbei...,100,100,78000,96000,80000,110000,"Based on national averages and online sources,..."
3,KAUFMÄNNISCHES PRAKTIKUM (100%) - MIT FOKUS KU...,gebana AG,Praktikum,DEINE AUFGABEN IM KUND:INNENSERVICE – EINSATZ ...,100,100,28300,28300,4500,6000,"Based on national averages and online sources,..."
4,System Engineer für Microsoft-Technologien und...,konekkt GmbH,Festanstellung,Wir präsentieren eine faszinierende Karrieremö...,80,100,95000,125000,90000,140000,"Based on national averages and online sources,..."


In [7]:
# all block1_answers where block1_max_salary is null and block1_min_salary is null
df_block1[df_block1['block1_max_salary'].isnull() & df_block1['block1_min_salary'].isnull()]


,title,company,contract_type,description,min_workload,max_workload,min_salary,max_salary,block1_min_salary,block1_max_salary,block1_answer


# Block 1 and Block 2 

In [ ]:
# change here for new prompt and message below ------------
results_file = 'data/llm_block1_2.csv'
min = 'block1_2_min_salary'
max = 'block1_2_max_salary'
answer = 'block1_2_answer'
# Setup and load initial data
if os.path.exists(results_file):
    df_without_salary = pd.read_csv(results_file)
else:
    df[min] = None  # Initialize columns if starting fresh
    df[max] = None
    df[answer] = None 

# change content here for new prompt and message below ------------
def generate_query(row):
    return {
        "role": "user",
        "content": f"Gib mir nur das Gehalt im Format (\d{1,3}(?:'\d{3})) und (\d{1,3}(?:'\d{3})) CHF für die Position {row['title']} bei der Firma {row['company']} in Zürich, Schweiz. Wenn du kein exaktes Gehalt hast gib eine Schätzung an."
    }

def run_model(limit):
    start_index = df[min].last_valid_index() + 1 if df[max].last_valid_index() != None else 0

    
    for index, row in df.iloc[start_index:start_index + limit].iterrows():
        query_message = generate_query(row)
        chat_completion = client.chat.completions.create(
            messages= [query_message], # change message for new prompt ------------
            model="llama3-70b-8192",
            temperature=0.0
        )
        min_salary, max_salary = extract_salaries(chat_completion.choices[0].message.content)
        print(index)
        print(min_salary, max_salary)
        
        
        # Update DataFrame
        df.at[index, min] = min_salary
        df.at[index, max] = max_salary
        df.at[index, answer] = chat_completion.choices[0].message.content
        
    # Save updated DataFrame in chunks or fully, depending on your preference
    df.to_csv(results_file, index=False)

# Testing the function with a limited number of entries
run_model(10000)

# LLM request with Block 1 2 3 

### overwrite regex pattern for block 1 2 3

In [8]:
# change here for new prompt and message below ------------
results_file = 'data/llm_block1_2_3.csv'
min = 'block1_2_3_min_salary'
max = 'block1_2_3_max_salary'
answer = 'block1_2_3_answer'
# Setup and load initial data
if os.path.exists(results_file):
    df = pd.read_csv(results_file)
else:
    df[min] = None  # Initialize columns if starting fresh
    df[max] = None
    df[answer] = None 

# change content here for new prompt and message below ------------
def generate_query(row):
    return {
        "role": "user",
        "content": f"Gib mir nur das Gehalt im Format (\d{1,3}(?:'\d{3})*) und (\d{1,3}(?:'\d{3})*) CHF für die Position {row['title']} bei der Firma {row['company']} in Zürich, Schweiz. Mit einer Arbeitszeit von {row['min_workload']} bis {row['max_workload']} Prozent. Wenn du kein exaktes Gehalt hast gib eine Schätzung an."
    }

def run_model(limit):
    start_index = df[min].last_valid_index() + 1 if df[max].last_valid_index() != None else 0

    
    for index, row in df.iloc[start_index:start_index + limit].iterrows():
        query_message = generate_query(row)
        chat_completion = client.chat.completions.create(
            messages= [query_message], # change message for new prompt ------------
            model="llama3-70b-8192",
            temperature=0.0
        )
        min_salary, max_salary = extract_salaries(chat_completion.choices[0].message.content)
        print(index)
        print(min_salary, max_salary)
        
        
        # Update DataFrame
        df.at[index, min] = min_salary
        df.at[index, max] = max_salary
        df.at[index, answer] = chat_completion.choices[0].message.content
        
    # Save updated DataFrame in chunks or fully, depending on your preference
    df.to_csv(results_file, index=False)

# Testing the function with a limited number of entries
run_model(10000)

0
80000 110000
1
70000 90000
2
80000 110000
3
4500 6000
4
90000 120000
5
None None
6
90000 140000
7
110000 140000
8
80000 110000
9
90000 120000
10
None None
11
80000 120000
12
110000 140000
13
80000 120000
14
120000 180000
15
None None
16
None None
17
80000 120000
18
80000 140000
19
90000 110000
20
90000 120000
21
90000 120000
22
90000 110000
23
90000 120000
24
80000 110000
25
80000 110000
26
80000 110000
27
90000 120000
28
90000 120000
29
90000 120000
30
80000 110000
31
90000 120000
32
95000 125000
33
90000 120000
34
90000 120000
35
110000 140000
36
90000 120000
37
None None
38
90000 120000
39
90000 120000
40
90000 120000
41
90000 120000
42
90000 120000
43
90000 120000
44
120000 150000
45
90000 120000
46
90000 120000
47
90000 120000
48
120000 150000
49
120000 180000
50
120000 150000
51
90000 120000
52
80000 110000
53
90000 120000
54
90000 120000
55
80000 120000
56
90000 120000
57
90000 120000
58
90000 120000
59
95000 125000
60
90000 120000
61
80000 110000
62
110000 140000
63
90000 120

In [9]:
df_block1_2_3 = pd.read_csv('data/llm_block1_2_3.csv')
df_block1_2_3.head()

,title,company,contract_type,description,min_workload,max_workload,min_salary,max_salary,block1_min_salary,block1_max_salary,block1_answer,block1_2_3_min_salary,block1_2_3_max_salary,block1_2_3_answer
0,Software Engineer .Net,ELCA Informatik AG,Festanstellung,"Über ELCA Wir sind ELCA, einer der grössten Sc...",80,100,100000,130000,80000,140000,"Based on national averages and online sources,...",80000.0,110000.0,"Based on national averages and online sources,..."
1,Digital Analyst / Web Analyst,Unic AG,Festanstellung,Deine Aufgaben als Digital Analyst bei Unic ...,80,100,90000,113000,80000,120000,"Based on national averages and online sources,...",70000.0,90000.0,"Based on national averages and online sources,..."
2,ICT Supporter 100% (a),Spitex Zürich,Festanstellung,Möchtest du auch etwas bewegen und deine Arbei...,100,100,78000,96000,80000,110000,"Based on national averages and online sources,...",80000.0,110000.0,"Based on national averages and online sources,..."
3,KAUFMÄNNISCHES PRAKTIKUM (100%) - MIT FOKUS KU...,gebana AG,Praktikum,DEINE AUFGABEN IM KUND:INNENSERVICE – EINSATZ ...,100,100,28300,28300,4500,6000,"Based on national averages and online sources,...",4500.0,6000.0,"Based on national averages and online sources,..."
4,System Engineer für Microsoft-Technologien und...,konekkt GmbH,Festanstellung,Wir präsentieren eine faszinierende Karrieremö...,80,100,95000,125000,90000,140000,"Based on national averages and online sources,...",90000.0,120000.0,"Based on national averages and online sources,..."


In [10]:
# all block1_answers where block1_max_salary is null and block1_min_salary is null
df_block1_2_3[df_block1_2_3['block1_2_3_max_salary'].isnull() & df_block1_2_3['block1_2_3_min_salary'].isnull()]


,title,company,contract_type,description,min_workload,max_workload,min_salary,max_salary,block1_min_salary,block1_max_salary,block1_answer,block1_2_3_min_salary,block1_2_3_max_salary,block1_2_3_answer
5,Senior Product Manager Domain Account Team,JobCloud AG,Festanstellung,The largest recruitment platform in Switzerlan...,100,100,112000,132000,120000,180000,"Based on national averages and online sources,...",NaN,NaN,"Based on national averages and online sources,..."
10,Senior Software Engineer (80% - 100%),JobCloud AG,Festanstellung,"At Jobcloud, we're not just a tech company, we...",80,100,104200,130800,120000,180000,"Based on national averages and online sources,...",NaN,NaN,"Based on national averages and online sources,..."
15,(Senior) Solution Architect .NET,N:SYNC AG,Festanstellung,"Einführung Wir, die N:SYNC AG, zählen zu den f...",100,100,110000,135000,120000,180000,"Based on national averages and online sources,...",NaN,NaN,"Based on national averages and online sources,..."
16,Manager Network and Infrastructure,Zattoo AG,Festanstellung,"YOUR FUTURE, ON DEMAND The ideal blend of sta...",100,100,130000,145000,120000,180000,"Based on national averages and online sources,...",NaN,NaN,"Based on national averages and online sources,..."
37,Kubernetes / Linux Engineer 80-100% (m/w/d),konekkt GmbH,Festanstellung,Bei unserem Kunden erwartet Dich eine spannend...,100,100,120000,140000,90000,120000,"Based on national averages and online sources,...",NaN,NaN,"Based on national averages and online sources,..."
79,System Engineer Azure Cloud 80-100% (m/w/d),konekkt GmbH,Festanstellung,Gestalte die Cloud Journey unseres führenden F...,100,100,105000,125000,90000,120000,"Based on national averages and online sources,...",NaN,NaN,"Based on national averages and online sources,..."
106,SAP Architect,TalentCloud Group GmbH,Festanstellung,Are you a highly motivated SAP Architect looki...,100,100,140000,180000,120000,180000,"Based on national averages and online sources,...",NaN,NaN,"Based on national averages and online sources,..."


# Block 1,2,3 and 4

In [ ]:
# change here for new prompt and message below ------------
results_file = 'data/llm_block1_2_3_4.csv'
min = 'block1_2_min_salary'
max = 'block1_2_max_salary'
answer = 'block1_2_answer'
# Setup and load initial data
if os.path.exists(results_file):
    df_without_salary = pd.read_csv(results_file)
else:
    df[min] = None  # Initialize columns if starting fresh
    df[max] = None
    df[answer] = None 

# change content here for new prompt and message below ------------
def generate_query(row):
    return {
        "role": "user",
        "content": f"Gib mir nur das Gehalt im Format (\d{1,3}(?:'\d{3})) und (\d{1,3}(?:'\d{3})) CHF für die Position {row['title']} bei der Firma {row['company']} in Zürich, Schweiz. Mit einer Arbeitszeit von {row['min_workload']} bis {row['max_workload']} Prozent und dem Vertragsart {row['contract_type']}. Wenn du kein exaktes Gehalt hast gib eine Schätzung an."
    }

def run_model(limit):
    start_index = df[min].last_valid_index() + 1 if df[max].last_valid_index() != None else 0

    
    for index, row in df.iloc[start_index:start_index + limit].iterrows():
        query_message = generate_query(row)
        chat_completion = client.chat.completions.create(
            messages= [query_message], # change message for new prompt ------------
            model="llama3-70b-8192",
            temperature=0.0
        )
        min_salary, max_salary = extract_salaries(chat_completion.choices[0].message.content)
        print(index)
        print(min_salary, max_salary)
        
        
        # Update DataFrame
        df.at[index, min] = min_salary
        df.at[index, max] = max_salary
        df.at[index, answer] = chat_completion.choices[0].message.content
        
    # Save updated DataFrame in chunks or fully, depending on your preference
    df.to_csv(results_file, index=False)

# Testing the function with a limited number of entries
run_model(10000)